In [ ]:
host = 'omero.hpc.virginia.edu' 
port = 4064
username = os.environ['USER']
group_id = -1
password = getpass.getpass()

# Connect

In [ ]:
import os
import getpass
import re

import omero
from omero.gateway import BlitzGateway
from omero.rtypes import rint, rlong, rstring, robject, unwrap

conn = BlitzGateway(username, password, host=host, port=port)
conn.connect()

# Import Parameters

In [ ]:
# fpath = '/scratch/khs3z/ORourke/JOW17/original'
#source
datasetId = 200
numCols = 12
rowLabels = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6, 'H':7,}
colLabels = {str(i+1).zfill(2):i for i in range(numCols)}

# target
screenId = 53

# Function to create Well Information for Image

In [ ]:
def createWell(conn,plateId,imageId):
    image = conn.getObject("Image",imageId)
    wellName = re.search('(?<=_Well)(.*?)(?=_Seq)', image.getName()).group(0)
    m = re.split('(\D+)', wellName)
    wellRow = rowLabels[m[1]]
    wellCol = colLabels[m[2]]
    print ("Row: %d, Col: %d, Well: %s, Name: %s" % (wellRow, wellCol, wellName, image.getName()))

    updateService = conn.getUpdateService()

    well = omero.model.WellI()
    well.plate = omero.model.PlateI(int(plateId), False)

    #wellRowCol = Well96_to_RowCol.get(wellNum)
    well.column = rint(wellCol) #rint(wellRowCol[1])
    well.row = rint(wellRow) #rint(wellRowCol[0])

    ws = omero.model.WellSampleI()
    ws.image = omero.model.ImageI(image.id,False)
    ws.well = well  
    well.addWellSample(ws)
    updateService.saveAndReturnObject(ws)

# Create a new Plate

In [ ]:
# Get the dataset
ds = conn.getObject('Dataset', datasetId)

# get service instance
updateService = conn.getUpdateService()

# Define new plate using the dataset name
plate = omero.model.PlateI()
plate.name = omero.rtypes.RStringI(ds.getName())
plate.columnNamingConvention = rstring("number")
plate.rowNamingConvention = rstring("letter")
plate = updateService.saveAndReturnObject(plate)
plateId = plate.id.val
plate = conn.getObject("Plate",plateId)


In [ ]:
# Get Images from Raw Dataset and create Well info
for img in ds.listChildren():
    createWell(conn, plateId, img.getId())

# Close Connection

In [ ]:
conn.close()

# Get files

In [ ]:
from os import listdir
from os.path import isfile, join


imgfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
for f in imgfiles:
    print f